# SIMP Topology Optimization using Gridap in Julia
Done by Mayank Shekhar

## Load required packages

In [1]:
# FEA 
using  Gridap
using  Gridap.Geometry
using  Gridap.Fields
using  Gridap.TensorValues 
using  Gridap.CellData 

# Meshing 
using  Gmsh 
using  GridapGmsh

using  LinearAlgebra 

# Gardient calculation 
using  ChainRulesCore, Zygote
import ChainRulesCore: rrule 

# For MMA 
using  NLopt 

# For plotting 
using CairoMakie, GridapMakie 

## Constant parameters
Material properties, such as modulus of elasticity and Poisson's ratio. The SIMP penalty is generally assumed as 3~5.

In [2]:
const  E_mat = 1.0
const  ν_mat = 0.3 
const  penal = 3; 

## Create the mesh or load already created mesh file using gmsh

In [3]:
# ################################################
# # Create the mesh 
# ################################################

const L = 80
const H = 120
const B = 40
const S = 60
const hf = L/40
const CW = H/12

gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)

gmsh.model.geo.addPoint(0.0, 0.0 , 0.0, hf ,1)
gmsh.model.geo.addPoint(L, 0.0, 0.0, hf, 2)
gmsh.model.geo.addPoint(L, H, 0.0, hf, 3)
gmsh.model.geo.addPoint(L-B, H-CW, 0.0, hf, 33)
gmsh.model.geo.addPoint(L-B, H, 0.0, hf, 4)
gmsh.model.geo.addPoint(L-B, H-S, 0.0, hf, 5)
gmsh.model.geo.addPoint(0, H-S, 0.0, hf, 6)

gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 33, 33)
gmsh.model.geo.addLine(33, 5, 4)
gmsh.model.geo.addLine(5, 6, 5)
gmsh.model.geo.addLine(6, 1, 6)

gmsh.model.geo.addCurveLoop([1,2,3,33,4,5,6],1)
gmsh.model.geo.addPlaneSurface([1], 1)

gmsh.model.addPhysicalGroup(2, [1],1)

gmsh.model.addPhysicalGroup(1, [5],1)
gmsh.model.addPhysicalGroup(1, [33],2)

gmsh.model.addPhysicalGroup(0, [1],1)
gmsh.model.addPhysicalGroup(0, [2],2)

gmsh.model.setPhysicalName(2, 1, "Domain")

gmsh.model.setPhysicalName(1, 1, "SittingPosition")
gmsh.model.setPhysicalName(1, 2, "HeadPosition")


gmsh.model.setPhysicalName(0, 1, "LeftSupport")
gmsh.model.setPhysicalName(0, 2, "RightSupport")

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)
gmsh.write("Chair.msh")
gmsh.finalize() 

#################################################
# Load the mesh 
#################################################
model = GmshDiscreteModel("Chair.msh"); 

#################################################
# Save the mesh in a vtk file 
#################################################
writevtk(model ,"Chair"); 

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 30%] Meshing curve 3 (Line)
Info    : [ 50%] Meshing curve 4 (Line)
Info    : [ 60%] Meshing curve 5 (Line)
Info    : [ 80%] Meshing curve 6 (Line)
Info    : [ 90%] Meshing curve 33 (Line)
Info    : Done meshing 1D (Wall 0.000351906s, CPU 0s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.04333s, CPU 0.046875s)
Info    : 2228 nodes 4461 elements
Info    : Writing 'Chair.msh'...
Info    : Done writing 'Chair.msh'
Info    : Reading 'Chair.msh'...
Info    : 15 entities
Info    : 2228 nodes
Info    : 4281 elements
Info    : Done reading 'Chair.msh'


### Constitutive matrix (1 for plane stress, 2 for plane strain) 
Elasticity tensor, constitutive relation, and SIMP penalty relation are formulated here. 

In [4]:
function  ElasFourthOrderConstTensor(E,ν,PlanarState)
    # 1 for  Plane  Stress  and 2 Plane  Strain  Condition
    if  PlanarState  == 1
        C1111 =E/(1-ν*ν)
        C1122 = (ν*E)/(1-ν*ν)
        C1112 = 0.0
        C2222 =E/(1-ν*ν)
        C2212 = 0.0
        C1212 =E/(2*(1+ν))
    elseif  PlanarState  == 2
        C1111 = (E*(1-ν*ν))/((1+ν)*(1-ν-2*ν*ν))
        C1122 = (ν*E)/(1-ν-2*ν*ν)
        C1112 = 0.0
        C2222 = (E*(1-ν))/(1-ν-2*ν*ν)
        C2212 = 0.0
        C1212 =E/(2*(1+ν))
    end
    C_ten = SymFourthOrderTensorValue(C1111 ,C1112 ,C1122 ,C1112 ,C1212 ,C2212 ,C1122 ,C2212 ,C2222)
    return   C_ten
end 

function σfun(ε)
    σ = C_mat⊙ε
    return  σ
end 

function Em(p)
    Em = p ^ penal
    return Em
end 

const  C_mat = ElasFourthOrderConstTensor(E_mat ,ν_mat ,1);

### Finite Element Analysis 

In [5]:
order = 1
reffe_Disp = ReferenceFE(lagrangian ,VectorValue{2,Float64},order)
V0_Disp = TestFESpace(model,reffe_Disp;conformity =:H1,
    dirichlet_tags = ["LeftSupport","RightSupport"],
    dirichlet_masks =[(true,true),(true,true)])
uh = zero(V0_Disp)
U0_Disp = V0_Disp

UnconstrainedFESpace()

In [6]:
degree = 2*order
Ω= Triangulation(model)
dΩ= Measure(Ω,degree) 

GenericMeasure()

In [7]:
labels = get_face_labeling(model)
LoadTagId1 = get_tag_from_name(labels ,"SittingPosition")
Γ_Load1 = BoundaryTriangulation(model ;tags = LoadTagId1)
dΓ_Load1 = Measure(Γ_Load1 ,degree)
n_Γ_Load1 = get_normal_vector(Γ_Load1)

GenericCellField():
 num_cells: 20
 DomainStyle: ReferenceDomain()
 Triangulation: BoundaryTriangulation()
 Triangulation id: 4976905946008182856

In [8]:
labels = get_face_labeling(model)
LoadTagId2 = get_tag_from_name(labels ,"HeadPosition")
Γ_Load2 = BoundaryTriangulation(model ;tags = LoadTagId2)
dΓ_Load2 = Measure(Γ_Load2 ,degree)
n_Γ_Load2 = get_normal_vector(Γ_Load2)

GenericCellField():
 num_cells: 5
 DomainStyle: ReferenceDomain()
 Triangulation: BoundaryTriangulation()
 Triangulation id: 17004754003014019102

In [9]:
p_reffe = ReferenceFE(lagrangian, Float64, 0)
Q = TestFESpace(Ω, p_reffe, vector_type = Vector{Float64})
P = Q
np = num_free_dofs(P) 

4254

In [10]:
pf_reffe = ReferenceFE(lagrangian, Float64, 1)
Qf = TestFESpace(Ω, pf_reffe, vector_type = Vector{Float64})
Pf = Qf

UnconstrainedFESpace()

In [11]:
fem_params = (;V0_Disp, U0_Disp, Q, P, Qf, Pf, np, Ω, dΩ, n_Γ_Load1, n_Γ_Load2, dΓ_Load1, dΓ_Load2) 

(V0_Disp = UnconstrainedFESpace(), U0_Disp = UnconstrainedFESpace(), Q = UnconstrainedFESpace(), P = UnconstrainedFESpace(), Qf = UnconstrainedFESpace(), Pf = UnconstrainedFESpace(), np = 4254, Ω = BodyFittedTriangulation(), dΩ = GenericMeasure(), n_Γ_Load1 = GenericCellField(), n_Γ_Load2 = GenericCellField(), dΓ_Load1 = GenericMeasure(), dΓ_Load2 = GenericMeasure())

In [12]:
A_Disp(u,v,pth) =  ((p->Em(p))∘pth) * (ε(v) ⊙ (σfun∘(ε(u))))
function MatrixA(pth; fem_params)
    A_mat = assemble_matrix(fem_params.U0_Disp, fem_params.V0_Disp) do u, v
        ∫(A_Disp(u,v,pth))fem_params.dΩ
    end
    return lu(A_mat)
end

MatrixA (generic function with 1 method)

In [13]:
f = VectorValue(0,-1.0) # Load vector 
g = VectorValue(0.7071,-0.7071)

VectorValue{2, Float64}(0.7071, -0.7071)

In [14]:
function  stepDisp(fem_params,pth)
    A_Disp(u,v,pth) = ((p->Em(p))∘pth) * ε(v) ⊙ (σfun∘(ε(u)))
    a_Disp(u,v) = ∫(A_Disp(u,v,pth))fem_params.dΩ
    b_Disp(v) = ∫(v ⋅ f)fem_params.dΓ_Load1 + ∫(v ⋅ g)fem_params.dΓ_Load2
    op_Disp = AffineFEOperator(a_Disp ,b_Disp ,fem_params.U0_Disp ,fem_params.V0_Disp)
    uh_out = solve(op_Disp)
    return  get_free_dof_values(uh_out)
end 

stepDisp (generic function with 1 method)

In [15]:
function MatrixOf(fem_params)

    return assemble_matrix(fem_params.U0_Disp, fem_params.V0_Disp) do u, v
             0.5*∫((∇(u))' ⊙ (C_mat ⊙ ∇(v)))fem_params.dΩ
    end
end
@show size(MatrixOf(fem_params)); 

size(MatrixOf(fem_params)) = (4452, 4452)


### Filtering and thresholding 
Helmholtz PDE-based filter is used here. 

(see: Lazarov, Boyan Stefanov, and Ole Sigmund. "Filters in topology optimization based on Helmholtz‐type differential equations." International Journal for Numerical Methods in Engineering 86.6 (2011): 765-781.) 

A smooth approximation of the Heaviside step function is used as the thresholding function. $\beta$ determines the strength of the approximation, and $\eta$ determines the location. Note that $\eta=0.5$ is standard in TO. 

In [16]:
elemsize = 2.0 # element size = hf = 80/40
r = (3*elemsize)/(2*sqrt(3)) #0.025           # Filter radius
β = 4                       # β∈[1,∞], threshold sharpness
η = 0.5                     # η∈[0,1], threshold center

a_f(r, u, v) = r^2 * (∇(v) ⋅ ∇(u))

function Filter(p0; r, fem_params)
    ph = FEFunction(fem_params.P, p0)
    op = AffineFEOperator(fem_params.Pf, fem_params.Qf) do u, v
        ∫(a_f(r, u, v))fem_params.dΩ + ∫(v * u)fem_params.dΩ, ∫(v * ph)fem_params.dΩ
      end
    pfh = solve(op)
    return get_free_dof_values(pfh)
end

function Threshold(pfh; β, η)
    return  ((tanh(β * η) + tanh(β * (pfh - η))) / (tanh(β * η) + tanh(β * (1.0 - η)))) 

end 

NO_FIELDS = ZeroTangent() 

Dptdpf(pf, β, η) = β * (1.0 - tanh(β * (pf - η))^2) / (tanh(β * η) + tanh(β * (1.0 - η))) # Gradient of thresholding function 
DEdpf(pf, β, η)= penal * ((Threshold(pf; β, η)) ^ (penal-1)) * Dptdpf(pf, β, η) # Gradient of density^penal
DAdpf(u, v, pfh; β, η) = ((p->DEdpf(p, β, η)) ∘ pfh) * (ε(v) ⊙ (σfun∘(ε(u)))); 

### FEA result with the entire domain filled with density 1 elements 
For checking!

In [17]:
# Comment/Uncomment for FEA Analysis 
p0 = ones(fem_params.np)
pf_vec = Filter(p0;r, fem_params)
pfh = FEFunction(fem_params.Pf, pf_vec)
pth = (pf -> Threshold(pf; β, η)) ∘ pfh
A_mat = MatrixA(pth; fem_params)
u_vec = stepDisp(fem_params,pth)
uh = FEFunction(fem_params.U0_Disp, u_vec)

writevtk(Ω,"FEAresultschair",cellfields=["uh"=>uh,"p0"=>p0,"pth"=>pth]);

### Gradient calculation 
Using adjoint method 

In [18]:
function gf_pf(pf_vec; β, η, fem_params)
    pfh = FEFunction(fem_params.Pf, pf_vec)
    pth = (pf -> Threshold(pf; β, η)) ∘ pfh
    u_vec = stepDisp(fem_params,pth)
    K_mat = MatrixOf(fem_params)
    u_vec' * K_mat * u_vec
end 

function rrule(::typeof(gf_pf), pf_vec; β, η, fem_params)
    function U_Disp_pullback(dg)
      NO_FIELDS, dg * Dgfdpf(pf_vec; β, η, fem_params) 
    end
    gf_pf(pf_vec; β, η, fem_params), U_Disp_pullback
end

function Dgfdpf(pf_vec; β, η, fem_params)
    pfh = FEFunction(fem_params.Pf, pf_vec)
    pth = (pf -> Threshold(pf; β, η)) ∘ pfh
    A_mat = MatrixA(pth; fem_params)
    u_vec = stepDisp(fem_params,pth)
    O_mat = MatrixOf(fem_params)
    uh = FEFunction(fem_params.U0_Disp, u_vec)
    w_vec =  A_mat' \ (O_mat * u_vec)
    wconjh = FEFunction(fem_params.U0_Disp, w_vec)
    l_temp(dp) = ∫(-2*DAdpf(wconjh,uh, pfh; β, η) * dp)fem_params.dΩ
    dgfdpf = assemble_vector(l_temp, fem_params.Pf)

    return dgfdpf
end 

function pf_p0(p0; r, fem_params)
    pf_vec = Filter(p0; r, fem_params)
    pf_vec
end

function rrule(::typeof(pf_p0), p0; r, fem_params)
  function pf_pullback(dgdpf)
    NO_FIELDS, Dgdp(dgdpf; r, fem_params)
  end
  pf_p0(p0; r, fem_params), pf_pullback
end

function Dgdp(dgdpf; r, fem_params)
    Af = assemble_matrix(fem_params.Pf, fem_params.Qf) do u, v
        ∫(a_f(r, u, v))fem_params.dΩ + ∫(v * u)fem_params.dΩ
    end
    wvec = Af' \ dgdpf
    wh = FEFunction(fem_params.Pf, wvec)
    l_temp(dp) = ∫(wh * dp)fem_params.dΩ
    return assemble_vector(l_temp, fem_params.P)
end 

function gf_p(p0::Vector; r, β, η, fem_params)
    pf_vec = pf_p0(p0; r, fem_params)
    gf_pf(pf_vec; β, η, fem_params)
end

function gf_p(p0::Vector, grad::Vector; r, β, η, fem_params)
    if length(grad) > 0
        dgdp, = Zygote.gradient(p -> gf_p(p; r, β, η, fem_params), p0)
        grad[:] = dgdp
    end
    gvalue = gf_p(p0::Vector; r, β, η, fem_params)
    open("gvaluemma.txt", "a") do io
        write(io, "$gvalue \n")
    end
    gvalue
end; 

### Volume constraint calculation 
Volume should be estimated with filtered densities and not with thresholded densities. 

In [19]:
volfrac = 0.3 

dv=get_array(∫(1)fem_params.dΩ)
getpoints = get_cell_points(Ω)

gradc = zeros(fem_params.np)
function cf_p(p0::Vector, gradc::Vector; r, β, η, fem_params)
    if length(gradc) > 0
        gradc[:] = dv
    end
    pf_vec = pf_p0(p0; r, fem_params)
    pfh = FEFunction(fem_params.Pf, pf_vec)
    pth = (pf -> Threshold(pf; β, η)) ∘ pfh
    pthxtr = evaluate(pfh⊙dv,getpoints)
    return sum(pthxtr)[1] - volfrac*sum(dv)
end; 

### MMA to optimize 
Svanberg, Krister. "The method of moving asymptotes—a new method for structural optimization." International journal for numerical methods in engineering 24.2 (1987): 359-373. 

In [20]:
function gf_p_optimize(p_init; r, β, η, TOL=1e-4, MAX_ITER, fem_params)
    ##################### Optimize #################
    opt = Opt(:LD_MMA, fem_params.np)
    opt.lower_bounds = 0.001
    opt.upper_bounds = 1
    opt.xtol_rel = TOL
    opt.maxeval = MAX_ITER
    opt.min_objective = (p0, grad) -> gf_p(p0, grad; r, β, η, fem_params)
    inequality_constraint!(opt, (p0, gradc) -> cf_p(p0, gradc; r, β, η, fem_params), 1e-8)
    (g_opt, p_opt, ret) = optimize(opt, p_init)
    @show numevals = opt.numevals # the number of function evaluations
    println("got $g_opt at $p_opt after $numevals iterations (returned $ret)")
    return g_opt, p_opt

end; 

## Run the MMA

In [21]:
# p0 = rand(fem_params.np) # For randomized densities 
grad = zeros(fem_params.np)

p_opt = fill(volfrac, fem_params.np)   # Uniform Initial guess
g_opt = 0

TOL = 1e-5 # tolerance of the MMA 
MAX_ITER = 4000 # Maximum iteration constraints 

g_opt, p_opt = gf_p_optimize(p_opt; r, β, η, TOL, MAX_ITER, fem_params);

###############################################
# For a list of β in the thresholding function 
#----------------------------------------------
# β_list = [4.0,8.0,16.0] 
# for bi = 1 : 3
#     β = β_list[bi]
#     g_opt, p_temp_opt = gf_p_optimize(p_opt; r, β, η, fem_params)
#     global p_opt = p_temp_opt
# end 
# @show g_opt 

numevals = opt.numevals = 1962
got 29013.943837986593 at [1.0, 0.001, 0.001, 0.24538051798223903, 0.001, 0.001, 0.001, 0.9145730279383872, 0.3457817899639117, 0.001, 1.0, 0.9123336369720562, 0.001, 0.8654996280343303, 0.001, 0.001, 0.001, 0.001, 0.014907956340318345, 0.001, 0.001, 0.001, 0.04947848169942228, 1.0, 0.8731815671988014, 0.001, 0.813343800828855, 1.0, 0.0391391841250209, 0.001, 0.001, 1.0, 1.0, 0.001, 1.0, 0.001, 1.0, 1.0, 0.001, 0.001, 1.0, 0.9614197411148634, 1.0, 0.94256119010994, 0.002863333900770439, 0.9953448046688016, 0.001, 1.0, 0.9887572565468786, 0.001, 0.001, 1.0, 0.03547650650969976, 1.0, 0.001, 1.0, 1.0, 0.797855918018064, 0.001, 0.001, 0.8591475938350125, 0.001, 0.001, 0.001, 0.001, 0.001, 0.03553713997636261, 0.001, 0.001, 0.003931551507876359, 0.001, 0.27858531536451436, 0.001, 0.03791104098777175, 0.004321960300670086, 0.0010006903690480125, 0.0013460786894531325, 0.0011712075555411745, 0.001, 1.0, 0.001, 1.0, 1.0, 0.001, 0.09634020677598071, 0.001, 0.001, 

### Postprocessing 
Redefine the thresholding function with a sharp increase (i.e., increase β) to get a crisp boundary 

In [22]:
βpost=64
function Thresholdp(pfh; βpost, η)
    return ((tanh(βpost * η) + tanh(βpost * (pfh - η))) / (tanh(βpost * η) + tanh(βpost * (1.0 - η)))) 
end 

pf_vec = pf_p0(p_opt; r, fem_params)
pfh = FEFunction(fem_params.Pf, pf_vec)
pth = (pf -> Thresholdp(pf; βpost, η)) ∘ pfh; 

### Save the optimized design in png and vtk files 

In [23]:
fig, ax, plt = plot(fem_params.Ω, pth, colormap = :binary)
Colorbar(fig[1,2], plt)
ax.aspect = AxisAspect(3)
ax.title = "Optimized Design"
# rplot = 110 # Region for plot
limits!(ax, 0, 80, 0, 120)
save("shapeChair.png", fig)

writevtk(Ω,"mayankChair",cellfields= ["p_opt"=>p_opt,"pfh"=>pfh,"pth"=>pth]); 